<a href="https://colab.research.google.com/github/BillWorstell/TruthInTranslation/blob/main/TranslateModels.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Below is a **minimal-change** solution demonstrating three fixes/updates:

1. **Omit the `system` role** (and optionally `temperature`) for models that don’t allow them.  
2. **Properly skip** any models whose sheets already exist.  
3. **Log execution time** for each model in addition to cost.  

The rest of the code is unchanged, so it *should* preserve your successful outputs in the workbook and only run new models (or newly added ones) that don’t already have a sheet.

---

### Snippet

```python
import os
import time
import openai
import pandas as pd
from datetime import datetime

###############################################################################
# 0) SETUP: Provide your key, model list, cost table, usage stats, etc.
###############################################################################
openai.api_key = "YOUR_OPENAI_API_KEY"

# You can use your existing model list
models_to_run = [
    "gpt-4o",
    "gpt-4o-mini",
    "o3-mini",
    "o1-mini",
    # plus any other older models if you want
]

# Price table (per 1M tokens) for your models (same as before)
COST_TABLE = {
    "gpt-4o":       {"input": 2.50/1_000_000,  "cached_input": 1.25/1_000_000, "output": 10.00/1_000_000},
    "gpt-4o-mini":  {"input": 0.15/1_000_000,  "cached_input": 0.075/1_000_000,"output": 0.60/1_000_000},
    "o3-mini":      {"input": 1.10/1_000_000,  "cached_input": 0.55/1_000_000, "output": 4.40/1_000_000},
    "o1-mini":      {"input": 1.10/1_000_000,  "cached_input": 0.55/1_000_000, "output": 4.40/1_000_000},
    # etc.
}

# For usage/cost tracking per model
USAGE_STATS = {m: {"prompt_tokens": 0, "completion_tokens": 0, "cost_usd": 0.0} for m in models_to_run}

# Two sets for checking whether a model supports system-role or temperature
MODELS_THAT_ALLOW_SYSTEM = ("gpt-3.5", "gpt-4")  # Extend if needed
MODELS_THAT_ALLOW_TEMPERATURE = ("gpt-3.5", "gpt-4")  # Extend if needed


###############################################################################
# 1) Usage Stats Helpers (unchanged except for the “system role” check)
###############################################################################
def update_usage_stats(response):
    model_used = response.model  # e.g. "o3-mini-2025-03-01"
    usage = response.usage
    pt = usage.prompt_tokens
    ct = usage.completion_tokens

    for short_name in COST_TABLE.keys():
        if model_used.startswith(short_name):
            USAGE_STATS[short_name]["prompt_tokens"] += pt
            USAGE_STATS[short_name]["completion_tokens"] += ct
            cost_in = pt * COST_TABLE[short_name]["input"]
            cost_out = ct * COST_TABLE[short_name]["output"]
            USAGE_STATS[short_name]["cost_usd"] += (cost_in + cost_out)
            break

def print_usage_stats():
    print("\n=== USAGE & COST SUMMARY ===")
    total = 0.0
    for m, st in USAGE_STATS.items():
        if st["prompt_tokens"] or st["completion_tokens"]:
            print(f"Model: {m}")
            print(f"  Prompt Tokens: {st['prompt_tokens']} | Completion Tokens: {st['completion_tokens']}")
            print(f"  Subtotal Cost: ${st['cost_usd']:.4f}\n")
            total += st["cost_usd"]
    print(f"OVERALL COST: ${total:.4f}")
    print("===========================\n")


###############################################################################
# 2) Translation Function: Minimal changes
###############################################################################
def translate_akan_to_english(model_name, text):
    """
    Calls the OpenAI ChatCompletion API to translate the given Akan text
    into English using the specified model. Includes a detailed footnote.
    """

    footnote_instruction = (
        "Please provide an expanded footnote for the Akan line you are translating. "
        "The footnote should include:\n"
        "1. Literal Translation Mapping\n"
        "2. Cultural Context\n"
        "3. Translation Clarification\n"
    )

    # For models that do not allow 'system' role, we pass everything as user content.
    supports_system_role = any(model_name.startswith(m) for m in MODELS_THAT_ALLOW_SYSTEM)
    supports_temperature = any(model_name.startswith(m) for m in MODELS_THAT_ALLOW_TEMPERATURE)

    if supports_system_role:
        # We can safely use role="system" and role="user".
        messages = [
            {"role": "system", "content": "You are a helpful translator from Akan to English."},
            {
                "role": "user",
                "content": (
                    f"Please translate the following Akan text to English and provide the footnote:\n"
                    f"{footnote_instruction}\n"
                    f"Akan Text: {text}"
                ),
            },
        ]
    else:
        # Combine all instructions in a single user message, no system role.
        combined_content = (
            "You are a helpful translator from Akan to English.\n\n"
            "Please translate the following Akan text to English and provide the footnote:\n"
            f"{footnote_instruction}\n"
            f"Akan Text: {text}"
        )
        messages = [{"role": "user", "content": combined_content}]

    # Build ChatCompletion kwargs
    api_kwargs = {
        "model": model_name,
        "messages": messages
    }
    if supports_temperature:
        # Some older ChatCompletion models support temperature=0
        api_kwargs["temperature"] = 0

    response = openai.ChatCompletion.create(**api_kwargs)
    update_usage_stats(response)
    return response["choices"][0]["message"]["content"].strip()


###############################################################################
# 3) Main Loop: skip existing sheets, measure time, etc.
###############################################################################
input_file = "/content/drive/My Drive/TruthInTranslation/Akan/Ananse3/AkanEnglishAligned.xlsx"
output_file = "/content/drive/My Drive/TruthInTranslation/Akan/Ananse3/Models.xlsx"

df_reference = pd.read_excel(input_file, sheet_name="Sheet1")
df_reference.columns = ["AKAN", "ENGLISH"]

# Get existing sheets (so we only run new models)
if os.path.exists(output_file):
    existing_sheets = pd.ExcelFile(output_file).sheet_names
else:
    existing_sheets = []

model_dataframes = {}
for model_name in models_to_run:
    if model_name in existing_sheets:
        print(f"Skipping model '{model_name}' - sheet already exists in {output_file}")
        continue

    print(f"Running model: {model_name}")
    time_start = time.time()

    df_model = df_reference.copy()
    df_model.columns = ["AKAN", model_name]

    # Translate each row
    for idx in range(len(df_model)):
        akan_text = df_model.loc[idx, "AKAN"]
        translated = translate_akan_to_english(model_name, akan_text)
        df_model.loc[idx, model_name] = translated

    model_dataframes[model_name] = df_model

    # Log time for just this model
    time_end = time.time()
    elapsed_sec = time_end - time_start
    print(f"Model '{model_name}' completed in {elapsed_sec:.2f} seconds.\n")


###############################################################################
# 4) Write new sheets, preserving old ones
###############################################################################
existing_dfs = {}
if os.path.exists(output_file):
    with pd.ExcelFile(output_file) as xls:
        for sheet in xls.sheet_names:
            existing_dfs[sheet] = pd.read_excel(xls, sheet_name=sheet)

with pd.ExcelWriter(output_file, engine="xlsxwriter") as writer:
    # Re-write old sheets
    for sheet, df in existing_dfs.items():
        df.to_excel(writer, sheet_name=sheet, index=False)

    # Add new model sheets
    for model_name, df_model in model_dataframes.items():
        df_model.to_excel(writer, sheet_name=model_name, index=False)

    # Optional: update a Metadata sheet if you want
    if "Metadata" not in existing_dfs:
        df_metadata = pd.DataFrame({
            "TimeStamp": [datetime.now().isoformat()],
            "ModelsUsed": [", ".join(models_to_run)],
            "NumLines": [len(df_reference)],
            "Notes": ["Detailed translations with expanded footnotes and time/cost logging."],
        })
        df_metadata.to_excel(writer, sheet_name="Metadata", index=False)

print(f"\nDone! Created or updated '{output_file}' with new model sheets.")
print_usage_stats()
```

---

### Explanation of the Three Key Fixes

1. **Omit “system” Role for Certain Models**  
   - We now check if the model’s name *starts with* `"gpt-3.5"` or `"gpt-4"`. If *not*, we bundle everything into the `"user"` role.  
   - This avoids “`Unsupported value: messages[0].role` does not support 'system'`” errors.  

2. **Properly Skip Already-Existing Sheets**  
   - Before running each model, we do:  
     ```python
     if model_name in existing_sheets:
         print(f"Skipping model '{model_name}' - sheet already exists")
         continue
     ```
   - This ensures we don’t re-run or overwrite a sheet that was successful on a previous run.  

3. **Log Execution Time**  
   - Right before we start calling `translate_akan_to_english`, we store `time_start = time.time()`.  
   - After finishing the loop for that model, we do `time_end = time.time()`.  
   - We print out how many seconds that model took: `elapsed_sec = time_end - time_start`.  

Everything else remains as close as possible to your previous code so that you keep your successful runs.

## Ensure you have the OpenAI Python library installed (pip install openai) and access to an OpenAI API key.

In [ ]:
#!pip install openai
!pip install openai==0.28
!pip install xlsxwriter

In [ ]:
import openai

#openai.api_key = "YOUR_OPENAI_API_KEY"


## Specify Input File

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
input_file = "/content/drive/My Drive/TruthInTranslation/Akan/Ananse10/AkanEnglishAligned.xlsx"
output_file = "/content/drive/My Drive/TruthInTranslation/Akan/Ananse10/Models.xlsx"

## Generate Output File

In [ ]:
import os
import time
import openai
import pandas as pd
from datetime import datetime

###############################################################################
# 0) SETUP: Provide your key, model list, cost table, usage stats, etc.
###############################################################################
#openai.api_key = "YOUR_OPENAI_API_KEY"

# You can use your existing model list
models_to_run = [
    "gpt-4o",
    "gpt-4o-mini",
    "o3-mini",
    "o1-mini",
    # plus any other older models if you want
]

# Price table (per 1M tokens) for your models (same as before)
COST_TABLE = {
    "gpt-4o":       {"input": 2.50/1_000_000,  "cached_input": 1.25/1_000_000, "output": 10.00/1_000_000},
    "gpt-4o-mini":  {"input": 0.15/1_000_000,  "cached_input": 0.075/1_000_000,"output": 0.60/1_000_000},
    "o3-mini":      {"input": 1.10/1_000_000,  "cached_input": 0.55/1_000_000, "output": 4.40/1_000_000},
    "o1-mini":      {"input": 1.10/1_000_000,  "cached_input": 0.55/1_000_000, "output": 4.40/1_000_000},
    # etc.
}

# For usage/cost tracking per model
USAGE_STATS = {m: {"prompt_tokens": 0, "completion_tokens": 0, "cost_usd": 0.0} for m in models_to_run}

# Two sets for checking whether a model supports system-role or temperature
MODELS_THAT_ALLOW_SYSTEM = ("gpt-3.5", "gpt-4")  # Extend if needed
MODELS_THAT_ALLOW_TEMPERATURE = ("gpt-3.5", "gpt-4")  # Extend if needed


###############################################################################
# 1) Usage Stats Helpers (unchanged except for the “system role” check)
###############################################################################
def update_usage_stats(response):
    model_used = response.model  # e.g. "o3-mini-2025-03-01"
    usage = response.usage
    pt = usage.prompt_tokens
    ct = usage.completion_tokens

    for short_name in COST_TABLE.keys():
        if model_used.startswith(short_name):
            USAGE_STATS[short_name]["prompt_tokens"] += pt
            USAGE_STATS[short_name]["completion_tokens"] += ct
            cost_in = pt * COST_TABLE[short_name]["input"]
            cost_out = ct * COST_TABLE[short_name]["output"]
            USAGE_STATS[short_name]["cost_usd"] += (cost_in + cost_out)
            break

def print_usage_stats():
    print("\n=== USAGE & COST SUMMARY ===")
    total = 0.0
    for m, st in USAGE_STATS.items():
        if st["prompt_tokens"] or st["completion_tokens"]:
            print(f"Model: {m}")
            print(f"  Prompt Tokens: {st['prompt_tokens']} | Completion Tokens: {st['completion_tokens']}")
            print(f"  Subtotal Cost: ${st['cost_usd']:.4f}\n")
            total += st["cost_usd"]
    print(f"OVERALL COST: ${total:.4f}")
    print("===========================\n")


###############################################################################
# 2) Translation Function: Minimal changes
###############################################################################
def translate_akan_to_english(model_name, text):
    """
    Calls the OpenAI ChatCompletion API to translate the given Akan text
    into English using the specified model. Includes a detailed footnote.
    """

    footnote_instruction = (
        "Please provide an expanded footnote for the Akan line you are translating. "
        "The footnote should include:\n"
        "1. Literal Translation Mapping\n"
        "2. Cultural Context\n"
        "3. Translation Clarification\n"
    )

    # For models that do not allow 'system' role, we pass everything as user content.
    supports_system_role = any(model_name.startswith(m) for m in MODELS_THAT_ALLOW_SYSTEM)
    supports_temperature = any(model_name.startswith(m) for m in MODELS_THAT_ALLOW_TEMPERATURE)

    if supports_system_role:
        # We can safely use role="system" and role="user".
        messages = [
            {"role": "system", "content": "You are a helpful translator from Akan to English."},
            {
                "role": "user",
                "content": (
                    f"Please translate the following Akan text to English and provide the footnote:\n"
                    f"{footnote_instruction}\n"
                    f"Akan Text: {text}"
                ),
            },
        ]
    else:
        # Combine all instructions in a single user message, no system role.
        combined_content = (
            "You are a helpful translator from Akan to English.\n\n"
            "Please translate the following Akan text to English and provide the footnote:\n"
            f"{footnote_instruction}\n"
            f"Akan Text: {text}"
        )
        messages = [{"role": "user", "content": combined_content}]

    # Build ChatCompletion kwargs
    api_kwargs = {
        "model": model_name,
        "messages": messages
    }
    if supports_temperature:
        # Some older ChatCompletion models support temperature=0
        api_kwargs["temperature"] = 0

    response = openai.ChatCompletion.create(**api_kwargs)
    update_usage_stats(response)
    return response["choices"][0]["message"]["content"].strip()


###############################################################################
# 3) Main Loop: skip existing sheets, measure time, etc.
###############################################################################
#input_file = "/content/drive/My Drive/TruthInTranslation/Akan/Ananse3/AkanEnglishAligned.xlsx"
#output_file = "/content/drive/My Drive/TruthInTranslation/Akan/Ananse3/Models.xlsx"

df_reference = pd.read_excel(input_file, sheet_name="Sheet1")
df_reference.columns = ["AKAN", "ENGLISH"]

# Get existing sheets (so we only run new models)
if os.path.exists(output_file):
    existing_sheets = pd.ExcelFile(output_file).sheet_names
else:
    existing_sheets = []

model_dataframes = {}
for model_name in models_to_run:
    if model_name in existing_sheets:
        print(f"Skipping model '{model_name}' - sheet already exists in {output_file}")
        continue

    print(f"Running model: {model_name}")
    time_start = time.time()

    df_model = df_reference.copy()
    df_model.columns = ["AKAN", model_name]

    # Translate each row
    for idx in range(len(df_model)):
        akan_text = df_model.loc[idx, "AKAN"]
        translated = translate_akan_to_english(model_name, akan_text)
        df_model.loc[idx, model_name] = translated

    model_dataframes[model_name] = df_model

    # Log time for just this model
    time_end = time.time()
    elapsed_sec = time_end - time_start
    print(f"Model '{model_name}' completed in {elapsed_sec:.2f} seconds.\n")


###############################################################################
# 4) Write new sheets, preserving old ones
###############################################################################
existing_dfs = {}
if os.path.exists(output_file):
    with pd.ExcelFile(output_file) as xls:
        for sheet in xls.sheet_names:
            existing_dfs[sheet] = pd.read_excel(xls, sheet_name=sheet)

with pd.ExcelWriter(output_file, engine="xlsxwriter") as writer:
    # Re-write old sheets
    for sheet, df in existing_dfs.items():
        df.to_excel(writer, sheet_name=sheet, index=False)

    # Add new model sheets
    for model_name, df_model in model_dataframes.items():
        df_model.to_excel(writer, sheet_name=model_name, index=False)

    # Optional: update a Metadata sheet if you want
    if "Metadata" not in existing_dfs:
        df_metadata = pd.DataFrame({
            "TimeStamp": [datetime.now().isoformat()],
            "ModelsUsed": [", ".join(models_to_run)],
            "NumLines": [len(df_reference)],
            "Notes": ["Detailed translations with expanded footnotes and time/cost logging."],
        })
        df_metadata.to_excel(writer, sheet_name="Metadata", index=False)

print(f"\nDone! Created or updated '{output_file}' with new model sheets.")
print_usage_stats()


Running model: gpt-4o
Model 'gpt-4o' completed in 198.61 seconds.

Running model: gpt-4o-mini
Model 'gpt-4o-mini' completed in 178.05 seconds.

Running model: o3-mini
Model 'o3-mini' completed in 439.29 seconds.

Running model: o1-mini
Model 'o1-mini' completed in 254.12 seconds.


Done! Created or updated '/content/drive/My Drive/TruthInTranslation/Akan/Ananse10/Models.xlsx' with new model sheets.

=== USAGE & COST SUMMARY ===
Model: gpt-4o
  Prompt Tokens: 5642 | Completion Tokens: 19383
  Subtotal Cost: $0.2079

Model: o3-mini
  Prompt Tokens: 2686 | Completion Tokens: 59041
  Subtotal Cost: $0.2627

Model: o1-mini
  Prompt Tokens: 2958 | Completion Tokens: 35928
  Subtotal Cost: $0.1613

OVERALL COST: $0.6320

